## 날씨 및 인구데이터 처리
(2019 기준)
- 기상청 날씨 데이터
- KOSIS 인구 데이터

In [85]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
print("pandas : ", pd.__version__)
print("numpy : ", np.__version__)

pandas :  1.0.5
numpy :  1.19.1


In [2]:
weather = pd.read_csv("weather.csv", encoding='cp949', engine='python')
weather.head()

,지점,지점명,일시,기온(°C),습도(%),일사(MJ/m^2),일조(Sec)
0,90,속초,2020-06-30 00:01,20.9,95.1,NaN,0.0
1,90,속초,2020-06-30 00:02,20.9,95.2,NaN,0.0
2,90,속초,2020-06-30 00:03,20.9,95.2,NaN,0.0
3,90,속초,2020-06-30 00:04,20.9,95.2,NaN,0.0
4,90,속초,2020-06-30 00:05,20.9,95.2,NaN,0.0


In [3]:
weather.isna().sum()

지점                0
지점명               0
일시                0
기온(°C)            6
습도(%)             2
일사(MJ/m^2)    57633
일조(Sec)        1440
dtype: int64

일사량과 일조량은 분석에 영향이 없을 것으로 판단되므로 제외하고 진행한다.\
기온과 습도에 결측치가 각각 6개, 2개 있는 것으로 보이는데, 결측이 발생한 이유를 살펴보기 위해 Null값이 있는 row를 구한다.

In [73]:
weather = weather.drop(['일사(MJ/m^2)', '일조(Sec)'], axis=1)

In [74]:
weather[pd.isnull(weather[['기온(°C)', '습도(%)']]).any(axis=1)]

,지점,지점명,일시,기온(°C),습도(%)
60465,177,홍성,2020-06-30 01:26,NaN,98.0
60466,177,홍성,2020-06-30 01:27,NaN,98.0
60467,177,홍성,2020-06-30 01:28,NaN,98.0
60468,177,홍성,2020-06-30 01:29,NaN,98.0
60469,177,홍성,2020-06-30 01:30,NaN,98.0
60470,177,홍성,2020-06-30 01:31,NaN,98.0
120568,272,영주,2020-06-30 19:14,21.3,NaN
120570,272,영주,2020-06-30 19:16,21.5,NaN


**홍성**에서 5분정도 되는 시간동안 기온이 측정되지 않고 **영주**에서 2분가량 습도가 측정되지 않은 것을 볼 수 있다.

- 기온과 습도가 분단위로 가변성이 크지 않을 것으로 판단되기 때문에 결측이 발생한 시간 전후 5개 정도의 데이터를 확인해본다

In [77]:
print("기온 결측")
display(weather[60460:60476])
print("습도 결측")
display(weather[120563:120576])

기온 결측


,지점,지점명,일시,기온(°C),습도(%)
60460,177,홍성,2020-06-30 01:21,19.9,98.0
60461,177,홍성,2020-06-30 01:22,19.9,98.0
60462,177,홍성,2020-06-30 01:23,19.9,98.0
60463,177,홍성,2020-06-30 01:24,19.9,98.0
60464,177,홍성,2020-06-30 01:25,19.9,98.0
60465,177,홍성,2020-06-30 01:26,NaN,98.0
60466,177,홍성,2020-06-30 01:27,NaN,98.0
60467,177,홍성,2020-06-30 01:28,NaN,98.0
60468,177,홍성,2020-06-30 01:29,NaN,98.0
60469,177,홍성,2020-06-30 01:30,NaN,98.0


습도 결측


,지점,지점명,일시,기온(°C),습도(%)
120563,272,영주,2020-06-30 19:09,20.5,86.8
120564,272,영주,2020-06-30 19:10,20.5,87.7
120565,272,영주,2020-06-30 19:11,20.5,86.8
120566,272,영주,2020-06-30 19:12,20.6,84.7
120567,272,영주,2020-06-30 19:13,20.7,76.1
120568,272,영주,2020-06-30 19:14,21.3,NaN
120569,272,영주,2020-06-30 19:15,21.7,65.9
120570,272,영주,2020-06-30 19:16,21.5,NaN
120571,272,영주,2020-06-30 19:17,21.1,81.6
120572,272,영주,2020-06-30 19:18,20.9,78.3


연속된 시간 혹은 비슷한 시간대에 결측이 일어났으므로 선형으로 비례하는 방식으로 결측값을 보간한다(interpolation)

> ### Pandas - interpolate()
> DataFrame 값에 선형으로 비례하는 방식으로 결측값 보간(method='values')
>> method='time'으로 설정할 경우 시계열 날짜 index를 기준으로 결측값 보간이 가능하다

In [80]:
w1 = weather.interpolate(method='values')
display(w1[60465:60471])
display(w1[120568:120570])

,지점,지점명,일시,기온(°C),습도(%)
60465,177,홍성,2020-06-30 01:26,19.914286,98.0
60466,177,홍성,2020-06-30 01:27,19.928571,98.0
60467,177,홍성,2020-06-30 01:28,19.942857,98.0
60468,177,홍성,2020-06-30 01:29,19.957143,98.0
60469,177,홍성,2020-06-30 01:30,19.971429,98.0
60470,177,홍성,2020-06-30 01:31,19.985714,98.0


,지점,지점명,일시,기온(°C),습도(%)
120568,272,영주,2020-06-30 19:14,21.3,71.0
120569,272,영주,2020-06-30 19:15,21.7,65.9


In [81]:
## 컬럼 명 변경 및 지점 코드 drop
w1 = w1.drop(['지점'], axis=1)
w1.columns = ['loc', 'time', 'temp', 'hum']
w1.head()

,loc,time,temp,hum
0,속초,2020-06-30 00:01,20.9,95.1
1,속초,2020-06-30 00:02,20.9,95.2
2,속초,2020-06-30 00:03,20.9,95.2
3,속초,2020-06-30 00:04,20.9,95.2
4,속초,2020-06-30 00:05,20.9,95.2


In [82]:
# 시도별 관측소 처리를 위한 사전
location = {'서울특별시':['관악산', '서울'],
            '부산광역시':['부산'],
            '대구광역시':['대구', '대구(기)'],
            '인천광역시':['강화', '백령도', '인천'],
            '광주광역시':['광주'],
            '대전광역시':['대전'],
            '울산광역시':['울산'],
            '경기도':['동두천', '수원', '양평', '이천', '파주'],
            '강원도':['강릉', '대관령', '동해', '북강릉', '북춘천', '삼척',
                   '속초', '영월', '원주', '인제', '정선군', '철원', '춘천',
                   '태백', '홍천'],
            '충청북도':['보은', '제천', '청주', '추풍령', '충주'],
            '충청남도':['금산', '보령', '부여', '서산', '천안', '홍성'],
            '전라북도': ['고창', '고창군', '군산', '남원', '부안', '순창군', '임실',
                     '장수', '전주', '정읍'],
            '전라남도':['강진군', '고흥', '광양시', '목포', '무안', '보성군',
                    '순천', '여수', '영광군', '완도', '장흥', '주암',
                    '진도(첨찰산)', '진도군', '해남', '흑산도'],
            '경상북도':['경주시', '구미', '문경', '봉화', '상주', '안동', '영덕',
                   '영주', '영천', '울릉도', '울진', '의성', '청송군', '포항'],
            '경상남도':['거제', '거창', '김해시', '남해', '밀양', '북창원', '산청',
                    '양산시', '의령군', '진주', '창원', '통영', '함양군', '합천'],
            '제주도':['고산', '서귀포', '성산', '성산포', '제주'],
            '세종특별자치시':['세종']
           }

In [90]:
def region(x):
    for key, value in location.items():
        if x in value:
            return key

In [91]:
w1['region'] = tqdm(w1['loc'].apply(region))

In [92]:
w1.isna().sum()

loc       0
time      0
temp      0
hum       0
region    0
dtype: int64

In [95]:
display(w1.head())
print(w1['region'].unique())

,loc,time,temp,hum,region
0,속초,2020-06-30 00:01,20.9,95.1,강원도
1,속초,2020-06-30 00:02,20.9,95.2,강원도
2,속초,2020-06-30 00:03,20.9,95.2,강원도
3,속초,2020-06-30 00:04,20.9,95.2,강원도
4,속초,2020-06-30 00:05,20.9,95.2,강원도


['강원도' '경기도' '인천광역시' '서울특별시' '경상북도' '충청북도' '충청남도' '대전광역시' '전라북도' '대구광역시'
 '울산광역시' '경상남도' '광주광역시' '부산광역시' '전라남도' '제주도' '세종특별자치시']


In [101]:
population = pd.read_csv("population.csv", encoding="cp949", engine="python", header=[0, 1, 2])
population

소재지(시군구)별(1)        2019                   
   소재지(시군구)별(1) 전체인구(A) (명)          도시지역 인구비율
   소재지(시군구)별(1)          소계 행정구역 인구기준(D/A*100)
0         서울특별시     9729107             100.00
1         부산광역시     3413841              99.50
2         대구광역시     2438031              99.13
3         인천광역시     2957026              97.80
4         광주광역시     1456468             100.00
5         대전광역시     1474870             100.00
6         울산광역시     1148019              95.26
7       세종특별자치시      340575              86.27
8           경기도    13239666              94.46
9           강원도     1541502              77.31
10         충청북도     1600007              79.36
11         충청남도     2123709              70.31
12         전라북도     1818917              78.82
13         전라남도     1868745              69.97
14         경상북도     2665836              74.85
15         경상남도     3362553              78.99
16      제주특별자치도      670989              94.49

## 인구 집계
- 도시지역 인구가 홈쇼핑 이용 비율이 높을 것
    - 총 인구 x 도시지역 인구비율
    - 집계, 지역별 도시지역 인구/총 도시지역 인구 -> 비율
    - 날씨 데이터에서 전국 가중합을 구할 때 가중치로 사용한다

In [102]:
population.columns = ['region', 'total', 'city']
population['mul'] = population['total']*population['city']
population.head()

,region,total,city,mul
0,서울특별시,9729107,100.00,9.729107e+08
1,부산광역시,3413841,99.50,3.396772e+08
2,대구광역시,2438031,99.13,2.416820e+08
3,인천광역시,2957026,97.80,2.891971e+08
4,광주광역시,1456468,100.00,1.456468e+08


In [111]:
pop1 = population.copy()
pop1['p'] = population['mul']/sum(population['mul'])
display(pop1)
print("sum of proportions : ", sum(pop1['p']))

,region,total,city,mul,p
0,서울특별시,9729107,100.00,9.729107e+08,0.205946
1,부산광역시,3413841,99.50,3.396772e+08,0.071903
2,대구광역시,2438031,99.13,2.416820e+08,0.051159
3,인천광역시,2957026,97.80,2.891971e+08,0.061217
4,광주광역시,1456468,100.00,1.456468e+08,0.030831
5,대전광역시,1474870,100.00,1.474870e+08,0.031220
6,울산광역시,1148019,95.26,1.093603e+08,0.023149
7,세종특별자치시,340575,86.27,2.938141e+07,0.006219
8,경기도,13239666,94.46,1.250619e+09,0.264732
9,강원도,1541502,77.31,1.191735e+08,0.025227


sum of proportions :  1.0


## Merge data
분당 데이터를 이용할 것이기 때문에 분단위로 나뉘어져 있는 형식은 우선 유지한다

In [113]:
w1.groupby(['time', 'region']).mean()

temp        hum
time             region                      
2020-06-30 00:01 강원도     19.864286  91.235714
                 경기도     19.800000  95.080000
                 경상남도    21.614286  96.042857
                 경상북도    19.671429  96.850000
                 광주광역시   22.500000  98.800000
...                            ...        ...
2020-07-01 00:00 전라남도    18.930769  88.723077
                 전라북도    18.350000  93.210000
                 제주도     19.800000  86.166667
                 충청남도    18.566667  87.150000
                 충청북도    17.740000  89.800000

[24432 rows x 2 columns]